# Transfer Learning Homework

## Question
- Your homework is to train on all digits
- make your own handwritten data set of 5 characters (i.e. A, B, C, D, E) 
- transfer your minist trained model over to them.

### make your own handwritten dataset of 5 characters (i.e. A, B, C, D, E). 
The English handwriting dataset will be used for this task:  
https://www.kaggle.com/dhruvildave/english-handwritten-characters-dataset

But this dataset contains handwritten numerical and all English alphabets. So we need to subest for the alphabets.



In [1]:

import datetime
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,  Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
import numpy as np
from keras.preprocessing import image
import PIL
import os


## Read in the new dataset. 
- It has a directory containing the handwritten alphabets.  
- And a label's file.  

In [370]:
path =r'.\imgs_char_zipped\imgs_char' 
#path = #'/content/drive/MyDrive/imgs_char/'
label= r'.\imgs_char_zipped\english.csv'

In [371]:
image_list=os.listdir(path)

In [372]:
import pandas as pd
#lbl_path= '/content/english.csv'
labels1 = pd.read_csv(label)
labels1

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0
...,...,...
3405,Img/img062-051.png,z
3406,Img/img062-052.png,z
3407,Img/img062-053.png,z
3408,Img/img062-054.png,z


# Match the label with the images list


## Extracting the filename from the 'image' array in the labels file.  
- We split along the '/' delimiter and return the last segment that is the image file name.

In [373]:
filename=[]
for i in range(len(labels1)):
  filename.append(labels1['image'][i].split('/')[-1])
  
labels1['filename']=filename

In [374]:
#Move the labels column to the last
labels1['label_']=labels1['label']

In [375]:
#Drop redundant column
labels1.drop(columns=['label'], inplace=True)

In [376]:
image_list = os.listdir(path)

## Handling the images dataset.  
- We use os.listdir() to access the list on images.
- We use load_img to read in each image.  
- Since the PIL function does not return the image as an array we will convert the returned PIL object to arrays of pixel values.
- We will reshape the output to 28, 28 that we expect later.  
- We put everything in a container.  


In [377]:
import PIL
import os

img_arrays=[]
image_lbl=[]
#images_dataset={}

for idx, i in enumerate(image_list):
    #print(i)
   #for j in range(len(labels1)):
    if i == labels1['filename'][idx]:
      real = os.path.join(path, i)
      image = tf.keras.utils.load_img(path = real, color_mode="grayscale", target_size=(28,28))
      #img_array = tf.keras.utils.img_to_array(img=image, dtype=np.uint8)
      img_array=tf.keras.preprocessing.image.img_to_array(img=image, dtype=np.uint8)
      img_array=np.reshape(img_array, (28,28))
      #print('idx==  ', labels1['filename'][idx], ' i== ', i)#, labels1['label'][idx])
      img_arrays.append(img_array)
      image_lbl.append(labels1['label_'][idx])

In [378]:
len(image_lbl)

3410

- Labels added to images.  


In [379]:
images_dataset = list(zip(image_lbl,img_arrays))
#images_dataset

In [380]:
#((images_dataset)[0])

### Extract only handwritten letters and exclude the handwritten digits.  


In [229]:
import collections
images_ds_ltr=[]
#images_dict_ltr={}
images_dict_ltr = collections.defaultdict(list)

for k,v in images_dataset:
    #print(v)
    if k in ['A', 'B', 'C', 'D', 'E']:
        #images_ds_ltr.append(images_dataset[i])
        images_dict_ltr[k].append(v)
print(len(images_dict_ltr))

5


In [382]:
from sklearn.model_selection import train_test_split
df_ds = {}
test_ltr=[]
train_ltr=[]
y_train3=[]
y_test3=[]
for k, v in images_dict_ltr.items():
    X_train1, X_test1 = train_test_split(v, test_size=0.33, random_state=42)
    df_ds[k]=(X_train1, X_test1) 
#for i in range(len(df_ds.items())):


for t_k, t_v in df_ds.items():
            #if t_k in list(df_ds.keys()) and :                
                train_ltr.extend(t_v[0]) 
 
                for i in range(len(t_v[0])):
                    y_train3.append(t_k)
                test_ltr.extend(t_v[1]) 
                for i in range(len(t_v[1])):
                    y_test3.append(t_k)


In [383]:
y_train3, y_test3, train_ltr, test_ltr = np.asarray(y_train3), np.asarray(y_test3), np.asarray(train_ltr), np.asarray(test_ltr)

In [384]:
train_ltr.shape

(180, 28, 28)

In [385]:
len(y_train3), len(y_test3), len(train_ltr), len(test_ltr)

(180, 95, 180, 95)

In [386]:
type(train_ltr)

numpy.ndarray

In [389]:
#Here we represent the categorical values with numerical codes.

y_train_code, y_train_uniques=pd.factorize(y_train3)
y_test_code, y_test_uniques = pd.factorize(y_test3)

In [390]:
#sanity check to see the items encoded. 
y_train_uniques, y_test_uniques

(array(['A', 'B', 'C', 'D', 'E'], dtype=object),
 array(['A', 'B', 'C', 'D', 'E'], dtype=object))

In [391]:
# see the codes
y_train_code

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4], dtype=int64)

In [392]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [393]:
# we need to visualize the shapes of the datasets going into the model
# It should be: number of instances, image height, image width

x_train.shape, train_ltr.shape, test_ltr.shape, y_train_code.shape, y_test_code.shape, y_train.shape

((60000, 28, 28), (180, 28, 28), (95, 28, 28), (180,), (95,), (60000,))

In [321]:
#new_train = np.vstack((x_train, train_ltr))
#new_test = np.vstack((x_test, test_ltr))
#new_y_train=np.concatenate((y_train,y_train_code))
#new_y_test=np.concatenate((y_test,y_test_code))

In [323]:
#new_train.shape, new_test.shape, new_y_train.shape, new_y_test.shape

((60180, 28, 28), (10095, 28, 28), (60180,), (10095,))

- We set important model fit hyperparameters here. 
- Kernel size of 7 is farely large but we will see how it performs.

In [369]:
now = datetime.datetime.now
batch_size = 128
num_classes = len(np.unique(y_train))
epochs = 15
img_rows, img_cols = 28, 28
filters = 64
pool_size = 2
kernel_size = 7


In [359]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)



# New Section

In [361]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    # convert class vectors to binary class matrices
    y_train = tf.keras.utils.to_categorical(train[1], num_classes)
    y_test = tf.keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])



## About the model.  
- The 1st layer uses the 64 7 X 7 filters and takes input shape 28 X 28.  
- The second layer uses the same number of filters and size. 
- We apply maxpooling after the second layer.  
- standard 'relu' activation was selected. 

**output layer**  
- These layers are flattened into a dense layer with 128 neurons.  
- The output layer uses 'softmax' activation and outputs 5 classes contained in the dataset.   


### Train on all 10 digits

In [362]:
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]


In [363]:
# create complete model
model = Sequential(feature_layers + classification_layers)

# train model for 5-digit classification [0..4]
train_model(model, [x_train, y_train],[x_test, y_test], num_classes)




x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Epoch 1/35
469/469 [==============================] - 63s 135ms/step - loss: 2.2839 - accuracy: 0.1570 - val_loss: 2.2472 - val_accuracy: 0.4084
Epoch 2/35
469/469 [==============================] - 61s 129ms/step - loss: 2.2287 - accuracy: 0.2726 - val_loss: 2.1718 - val_accuracy: 0.5654
Epoch 3/35
469/469 [==============================] - 62s 132ms/step - loss: 2.1499 - accuracy: 0.3661 - val_loss: 2.0563 - val_accuracy: 0.6316
Epoch 4/35
469/469 [==============================] - 61s 130ms/step - loss: 2.0291 - accuracy: 0.4344 - val_loss: 1.8836 - val_accuracy: 0.6901
Epoch 5/35
469/469 [==============================] - 62s 131ms/step - loss: 1.8657 - accuracy: 0.4860 - val_loss: 1.6554 - val_accuracy: 0.7440
Epoch 6/35
469/469 [==============================] - 61s 130ms/step - loss: 1.6765 - accuracy: 0.5301 - val_loss: 1.4072 - val_accuracy: 0.7769
Epoch 7/35
469/469 [==============================] - 62s

In [364]:
train_ltr.shape, test_ltr.shape, y_train_code.shape, y_test_code.shape

((180, 28, 28), (95, 28, 28), (180,), (95,))

In [365]:
num_classes= len(np.unique(y_train_code))
num_classes

5

## Output layer.  
- The layer has been to changed to 5 since we only have A, B, C, D, E in the new dataset.

In [366]:
classification_layers1 = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(5),
    Activation('softmax')
]

In [367]:
# freeze feature layers' weights and rebuild model
model = Sequential(feature_layers + classification_layers1)
#model1 = keras.Model(inputs=feature_layers.input, outputs=model)
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (train_ltr, y_train_code),
            (test_ltr, y_test_code), num_classes)

x_train shape: (180, 28, 28, 1)
180 train samples
95 test samples
Epoch 1/35
2/2 [==============================] - 0s 140ms/step - loss: 2.3188 - accuracy: 0.2222 - val_loss: 1.6959 - val_accuracy: 0.2526
Epoch 2/35
2/2 [==============================] - 0s 56ms/step - loss: 2.4216 - accuracy: 0.2222 - val_loss: 1.6890 - val_accuracy: 0.2632
Epoch 3/35
2/2 [==============================] - 0s 57ms/step - loss: 2.3918 - accuracy: 0.2389 - val_loss: 1.6817 - val_accuracy: 0.2526
Epoch 4/35
2/2 [==============================] - 0s 58ms/step - loss: 2.3234 - accuracy: 0.1667 - val_loss: 1.6755 - val_accuracy: 0.2526
Epoch 5/35
2/2 [==============================] - 0s 57ms/step - loss: 2.4153 - accuracy: 0.1500 - val_loss: 1.6698 - val_accuracy: 0.2421
Epoch 6/35
2/2 [==============================] - 0s 56ms/step - loss: 2.4230 - accuracy: 0.2167 - val_loss: 1.6635 - val_accuracy: 0.2526
Epoch 7/35
2/2 [==============================] - 0s 57ms/step - loss: 2.1568 - accuracy: 0.2389 - 

## Interpretation.  
Looking the training result above (Test Accuracy of 28%), the model architecture performed poorly on this dataset.  

In [368]:
# unfreeze feature layers and rebuild model to see the difference
model = Sequential(feature_layers + classification_layers1)
#model1 = keras.Model(inputs=feature_layers.input, outputs=model)
for l in feature_layers:
    l.trainable = True

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (train_ltr, y_train_code),
            (test_ltr, y_test_code), num_classes)

x_train shape: (180, 28, 28, 1)
180 train samples
95 test samples
Epoch 1/35
2/2 [==============================] - 1s 176ms/step - loss: 2.2208 - accuracy: 0.1833 - val_loss: 1.5674 - val_accuracy: 0.2842
Epoch 2/35
2/2 [==============================] - 0s 96ms/step - loss: 2.1855 - accuracy: 0.2222 - val_loss: 1.5643 - val_accuracy: 0.2842
Epoch 3/35
2/2 [==============================] - 0s 96ms/step - loss: 2.4249 - accuracy: 0.1667 - val_loss: 1.5617 - val_accuracy: 0.2842
Epoch 4/35
2/2 [==============================] - 0s 98ms/step - loss: 2.0500 - accuracy: 0.2333 - val_loss: 1.5579 - val_accuracy: 0.2842
Epoch 5/35
2/2 [==============================] - 0s 95ms/step - loss: 2.0957 - accuracy: 0.2167 - val_loss: 1.5559 - val_accuracy: 0.2842
Epoch 6/35
2/2 [==============================] - 0s 95ms/step - loss: 2.2555 - accuracy: 0.1778 - val_loss: 1.5537 - val_accuracy: 0.2842
Epoch 7/35
2/2 [==============================] - 0s 94ms/step - loss: 2.2606 - accuracy: 0.1833 - 

- When unfroze the layers, there was still no significant improvement to the accuracy result.  
- This suggests that the model architecture is not suitable for this dataset features.